In [29]:
library(mvtnorm)
library(matlib)
library(splitstackshape)

In [304]:
p <- 3
n.W1 <- 200
n.W2 <- 300
n <- n.W1 + n.W2

In [305]:
get_covariance_matrix <- function(n) {
    A <- matrix(runif(n^2) * 2 - 1, ncol = n) 
    Sigma <- t(A) %*% A
    
    return(Sigma)
}
get_means_vector <- function(n, a, b) {
    mu <- runif(n, a, b)
    
    return(mu)
}

## Well separable data

In [306]:
Sigma.W1 <- get_covariance_matrix(p)
Sigma.W2 <- Sigma.W1
Sigma.W1

0.5179821,0.16722670,-0.58226982
0.1672267,0.95793159,-0.09105213
-0.5822698,-0.09105213,0.67014613


In [307]:
mu.W1 <- get_means_vector(p, 4, 4.5)
mu.W2 <- get_means_vector(p, 1, 1.5)
mu.W1
mu.W2

[1] 4.360291 4.031501 4.230148

[1] 1.035993 1.030714 1.342504

In [308]:
X.W1 <- data.frame(rmvnorm(n.W1, mu.W1, Sigma.W1), class = matrix(1, n.W1, 1))
X.W2 <- data.frame(rmvnorm(n.W2, mu.W2, Sigma.W2), class = matrix(2, n.W2, 1))

In [309]:
q1 <- 0.4
q2 <- 1 - q1

n.test <- 100

### Create test data

In [310]:
X.test <- rbind(data.frame(rmvnorm(as.integer(q1 * n.test), mu.W1, Sigma.W1), 
                           class = matrix(1, as.integer(q1 * n.test), 1)),
                data.frame(rmvnorm(as.integer(q2 * n.test), mu.W2, Sigma.W2), 
                           class = matrix(2, as.integer(q2 * n.test), 1)))
X.test <- X.test[sample(nrow(X.test)), ]
X.test[0:5, ]

,X1,X2,X3,class
,<dbl>,<dbl>,<dbl>,<dbl>
4,4.9170563,4.8272296,3.6796267,1
80,-0.6996080,0.6834116,3.2081293,2
66,0.5794488,1.7966245,2.0081490,2
57,1.6777706,1.0999067,0.5733355,2
79,0.6236528,0.7843591,1.8890216,2


### Discriminant function

In [311]:
alpha <- inv(Sigma.W1) %*% (mu.W1 - mu.W2)
alpha

1431.4422
-129.8038
1230.4098


In [312]:
xi1 <- sum(alpha * mu.W1)
xi2 <- sum(alpha * mu.W2)

c <- (xi1 + xi2) / 2
c

[1] 6962.01

In [313]:
get_sigma_z_squared <- function(alpha, Sigma, p) {
    sigma_z.squared = 0
    for (m in (1:p)) {
        for (j in (1:p)) {
            sigma_z.squared = sigma_z.squared + alpha[m] * Sigma[m, j] * alpha[j]
        }
    } 
    return(sigma_z.squared)
}

In [314]:
sigma_z.squared <- get_sigma_z_squared(alpha, Sigma.W1, p)
sigma_z.squared

[1] 7922.012

In [315]:
dist.mhl <- (xi1 - xi2)^2 / sigma_z.squared
dist.mhl

[1] 7922.012

In [316]:
predict_class <- function(alpha, c, X, q1, q2, xi1, xi2) {
    predicted <- matrix(0, dim(X)[1], 1)
    
    for (i in 1:dim(X)[1]) {
        alpha_X <- sum(alpha * X[i,])
        predicted[i] = ifelse(alpha_X >= (xi1 + xi2) * 0.5 + log(q2 / q1), 1, 2)
    }
    
    return(predicted)
}

In [317]:
X.predict <- predict_class(alpha, c, X.test[,-4], q1, q2, xi1, xi2)

In [318]:
length(X.predict[X.predict == 1,])

[1] 40

In [319]:
length(X.predict[X.predict == 2,])

[1] 60

### Confusion table

In [320]:
get_conf_table <- function(data.predicted, deta.real) {
    conf_table.test <- data.frame(matrix(0, 2, 2))
    colnames(conf_table.test) <- c("1_pred", "2_pred")
    rownames(conf_table.test) <- c("1_real", "2_real")
    
    for (i in 1:2) {
        for (j in 1:2) {
            conf_table.test[i, j] <- sum(data.predicted[which(deta.real == i),] == j)
        }
    }
    
    return(conf_table.test)
    
}

In [321]:
get_conf_table(X.predict, X.test$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,40,0
2_real,0,60


In [322]:
X.W1.predict <- predict_class(alpha, c, X.W1[,-4], q1, q2, xi1, xi2)
get_conf_table(X.W1.predict, X.W1$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,200,0
2_real,0,0


In [323]:
X.W2.predict <- predict_class(alpha, c, X.W2[,-4], q1, q2, xi1, xi2)
get_conf_table(X.W2.predict, X.W2$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,0,0
2_real,0,300


## Badly separable data

In [324]:
Sigma.W1.bad <- get_covariance_matrix(p)
Sigma.W2.bad <- Sigma.W1.bad
Sigma.W1.bad

0.47568568,0.03056266,-0.5570420
0.03056266,1.94565722,-0.5415347
-0.55704198,-0.54153473,0.8044463


In [339]:
mu.W1.bad <- get_means_vector(p, 1, 1.8)
mu.W2.bad <- get_means_vector(p, 1, 1.5)
mu.W1.bad
mu.W2.bad

[1] 1.538475 1.566968 1.364361

[1] 1.365431 1.170656 1.245366

In [340]:
X.W1.bad <- data.frame(rmvnorm(n.W1, mu.W1.bad, Sigma.W1.bad), 
                       class = matrix(1, n.W1, 1))
X.W2.bad <- data.frame(rmvnorm(n.W2, mu.W2.bad, Sigma.W2.bad), 
                       class = matrix(2, n.W2, 1))

X.test.bad <- rbind(data.frame(rmvnorm(as.integer(q1 * n.test), mu.W1.bad, Sigma.W1.bad), 
                           class = matrix(1, as.integer(q1 * n.test), 1)),
                    data.frame(rmvnorm(as.integer(q2 * n.test), mu.W2.bad, Sigma.W2.bad), 
                           class = matrix(2, as.integer(q2 * n.test), 1)))

X.test.bad <- X.test.bad[sample(nrow(X.test.bad)), ]
X.test.bad[0:5, ]

,X1,X2,X3,class
,<dbl>,<dbl>,<dbl>,<dbl>
88,2.8694964,2.864340,-1.036206,2
51,0.9049486,1.218381,1.735854,2
70,1.7626245,-1.844569,1.582391,2
26,1.4149811,1.768614,1.765334,1
45,1.6255438,2.109315,0.531306,2


In [341]:
alpha.bad <- inv(Sigma.W1.bad) %*% (mu.W1.bad - mu.W2.bad)
alpha.bad

24.059076
5.542295
20.538669


In [342]:
xi1.bad <- sum(alpha.bad * mu.W1.bad)
xi2.bad <- sum(alpha.bad * mu.W2.bad)

c.bad <- (xi1.bad + xi2.bad) / 2
c.bad

[1] 69.31917

In [343]:
sigma_z.squared.bad <- get_sigma_z_squared(alpha.bad, Sigma.W1.bad, p)
sigma_z.squared.bad

[1] 8.80378

In [344]:
dist.mhl.bad <- (xi1.bad - xi2.bad)^2 / sigma_z.squared.bad
dist.mhl.bad

[1] 8.80378

In [345]:
X.predict.bad <- predict_class(alpha.bad, c.bad, 
                               X.test.bad[,-4], 
                               q1, q2, 
                               xi1.bad, xi2.bad)

In [346]:
get_conf_table(X.predict.bad, X.test.bad$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,38,2
2_real,2,58


In [347]:
X.W1.predict.bad <- predict_class(alpha.bad, c.bad, X.W1.bad[,-4], 
                                  q1, q2, xi1.bad, xi2.bad)
get_conf_table(X.W1.predict.bad, X.W1.bad$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,180,20
2_real,0,0


In [348]:
X.W2.predict.bad <- predict_class(alpha.bad, c.bad, X.W2.bad[,-4], 
                                  q1, q2, xi1.bad, xi2.bad)
get_conf_table(X.W2.predict.bad, X.W2.bad$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,0,0
2_real,19,281


### A'posteriori probabilities

##### Well separable data

In [335]:
K <- log(q2 / q1)
p_2_1 <- pnorm((K - 0.5 * dist.mhl) / sqrt(dist.mhl))
p_2_1

[1] 0

In [336]:
p_1_2 <- pnorm((-K - 0.5 * dist.mhl) / sqrt(dist.mhl))
p_1_2

[1] 0

##### Badly separable data

In [337]:
p_2_1.bad <- pnorm((K - 0.5 * dist.mhl.bad) / sqrt(dist.mhl.bad))
p_2_1.bad

[1] 0.0273474

In [338]:
p_1_2.bad <- pnorm((-K - 0.5 * dist.mhl.bad) / sqrt(dist.mhl.bad))
p_1_2.bad

[1] 0.01692468

## Real data
https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/

In [149]:
data.credit <- read.csv(file = 'data/german.data-numeric', header = FALSE, sep = "")
colnames(data.credit)[dim(data.credit)[2]] <- "class"
data.credit[, -25] <- scale(data.credit[, -25])
data.credit[1:5,]

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V16,V17,V18,V19,V20,V21,V22,V23,V24,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-1.2539382,-1.2358595,1.3433419,-0.7330651,1.8322523,1.3374093,0.4491018,1.0464631,-1.2930760,2.7650729,⋯,-0.5524288,-0.3386922,0.320052,-0.2066643,-0.4666999,0.6341309,-0.1499079,-0.4997499,0.7659728,1
-0.4587967,2.2470700,-0.5031762,0.9658932,-0.6993571,-0.3178002,-0.9631679,-0.7655942,-1.2930760,-1.1908081,⋯,-0.5524288,-0.3386922,0.320052,-0.2066643,-0.4666999,0.6341309,-0.1499079,-0.4997499,0.7659728,2
1.1314864,-0.7382981,1.3433419,-0.4145104,-0.6993571,0.5098045,0.4491018,0.1404344,-1.2930760,1.1827205,⋯,-0.5524288,-0.3386922,0.320052,-0.2066643,-0.4666999,0.6341309,-0.1499079,1.9989997,-1.3042239,1
-1.2539382,1.7495086,-0.5031762,1.6383976,-0.6993571,0.5098045,0.4491018,1.0464631,-0.3408845,0.8310866,⋯,-0.5524288,-0.3386922,-3.121368,-0.2066643,-0.4666999,-1.5753845,-0.1499079,-0.4997499,0.7659728,1
-1.2539382,0.2568246,0.4200829,0.5765486,-0.6993571,-0.3178002,0.4491018,1.0464631,1.5634983,1.5343544,⋯,1.8083779,-0.3386922,0.320052,-0.2066643,-0.4666999,-1.5753845,-0.1499079,-0.4997499,0.7659728,2


In [150]:
get_means_est <- function(data) {
    return(as.vector(apply(data, 2, mean)))
}

In [151]:
get_sigma_est <- function(data.1, data.2, p, mu.1, mu.2, n.1, n.2) {
    S.1 <- matrix(0, p, p)
    S.2 <- matrix(0, p, p)
    
    for (l in 1:p) {
        for (j in 1:p) {
            S.1[l, j] <- (1 / (n.1 - 1)) * sum((data.1[,l] - mu.1[l]) * (data.1[,j] - mu.1[j]))
            S.2[l, j] <- (1 / (n.2 - 1)) * sum((data.2[,l] - mu.2[l]) * (data.2[,j] - mu.2[j]))
        }
    }
    S <- (1 / (n.1 + n.2 - 2)) * ((n.1 - 1) * S.1 + (n.2 - 1) * S.2)
    
    return(S)
}

In [152]:
get_sigma_squared_est <- function(alpha, sigma, p) {
    sigma_squared = 0
    for (l in 1:p) {
        for (j in 1:p) {
            sigma_squared = sigma_squared + alpha[l] * sigma[l, j] * alpha[j]
        }
    }
    
    return(sigma_squared)
}

In [226]:
get_dist_mah <- function(xi.1, xi.2, sigma.squared, n.1, n.2, p) {
    dist.mah.skew <- (xi.1 - xi.2)^2 / sigma.squared
    dist.mah <- ((n.1 + n.2 - p - 3) / (n.1 + n.2 - 2)) * dist.mah.skew - p * ((1 / n.1) + (1 / n.2))

    return(dist.mah)
}

In [154]:
predict_class_est <- function(alpha, data, q1, q2, xi1, xi2) {
    predicted <- matrix(0, dim(data)[1], 1)
    
    for (i in 1:dim(data)[1]) {
        alpha_X <- sum(alpha * data[i,])
        predicted[i] = ifelse(alpha_X >= (xi1 + xi2) * 0.5 + log(q2 / q1), 1, 2)
    }
    
    return(predicted)
}

In [155]:
p <- dim(data.credit)[2] - 1
n <- dim(data.credit)[1]

##### Stratified data sampling

In [156]:
data.credit.test <- stratified(data.credit, "class", 0.2, keep.rownames = TRUE)
test.index <- apply(data.credit.test[, 1], 2, as.numeric)
data.credit.test[, 1] <- NULL
data.credit.train <- data.credit[-test.index,]

In [157]:
data.credit.train.1 <- data.credit.train[data.credit.train$class == 1,]
data.credit.train.2 <- data.credit.train[data.credit.train$class == 2,]

data.credit.test.1 <- data.credit.test[data.credit.test$class == 1,]
data.credit.test.2 <- data.credit.test[data.credit.test$class == 2,]

In [158]:
n.train.1 <- dim(data.credit.train.1)[1]
n.train.2 <- dim(data.credit.train.2)[1]
n.train <- n.train.1 + n.train.2

n.test.1 <- dim(data.credit.test.1)[1]
n.test.2 <- dim(data.credit.test.2)[1]
n.test <- n.test.1 + n.test.2

In [159]:
n.train
n.test

[1] 800

[1] 200

In [179]:
q1 <- n.train.1 / n.train
q2 <- n.train.2 / n.train
q1
q2

[1] 0.7

[1] 0.3

In [180]:
mu.1 <- get_means_est(data.credit.train.1[,-25])
mu.2 <- get_means_est(data.credit.train.2[,-25])

In [181]:
sigma <- get_sigma_est(data.credit.train.1, data.credit.train.2, p, mu.1, mu.2, n.train.1, n.train.2)

In [182]:
alpha <- inv(sigma) %*% (mu.1 - mu.2)

In [222]:
alpha_z.1 <- t(apply(data.credit.train.1[,-25], 1, function(x) alpha * x))
alpha_z.2 <- t(apply(data.credit.train.2[,-25], 1, function(x) alpha * x))
                     
z.1 <- as.matrix(apply(alpha_z.1, 1, sum))
z.2 <- as.matrix(apply(alpha_z.2, 1, sum))

In [223]:
xi.1 <- mean(z.1)
xi.2 <- mean(z.2)
xi.1
xi.2

[1] 0.5041788

[1] -1.175979

In [224]:
sigma.squared <- get_sigma_squared_est(alpha, sigma, p)
sigma.squared

[1] 1.680158

In [227]:
dist.mah <- get_dist_mah(xi.1, xi.2, sigma.squared, n.train.1, n.train.2, p)
dist.mah

[1] 1.484665

In [228]:
n.test

[1] 200

In [232]:
data.pred.test <- predict_class_est(alpha, data.credit.test[, -25], q1, q2, xi.1, xi.2)
get_conf_table(data.pred.test, data.credit.test$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,120,20
2_real,27,33


In [233]:
data.pred.train <- predict_class_est(alpha, data.credit.train[, -25], q1, q2, xi.1, xi.2)
get_conf_table(data.pred.train, data.credit.train$class)

,1_pred,2_pred
,<dbl>,<dbl>
1_real,507,53
2_real,111,129


In [236]:
K <- log(q2 / q1)

p_2_1 <- pnorm((K - 0.5 * dist.mah) / sqrt(dist.mah))
p_1_2 <- pnorm((-K - 0.5 * dist.mah) / sqrt(dist.mah))
p_2_1
p_1_2

[1] 0.09601222

[1] 0.5343246